# Polychromatic beam reconstruction

In this notebook, the pre-processed spray projected density images will be tomographically reconstructed into 3D volumes using the refined geometry parameters.

*Note*: this notebook requires a CUDA-capable GPU for 3D tomographic reconstruction using the *ASTRA* toolbox. For demonstration purposes, if no GPU is present (or the CUDA version is outdated), then the notebook will fall back to loading pre-computed results from the `/inputs` folder.

---

## Environment setup

In [1]:
import ipywidgets as widgets
import json
import numpy as np

from ipywidgets import (
                        fixed,
                        interact
                        )
from matplotlib import pyplot as plt

from recon_utilities import (
                             loadSpray,
                             getSystemMatrix,
                             MLOS,
                             scalarCF,
                             massCheck,
                             recon,
                             spatialCF,
                             showHistory,
                             compareProj,
                             convertVTK,
                             showVolume
                             )


# Data directories
inp_fld = 'inputs'
out_fld = 'outputs'

### Version information

In [2]:
%reload_ext watermark
%watermark -p ipywidgets,json,numpy,matplotlib,astra,h5py,pyvista
%watermark -p pyevtk,scipy,skimage

ipywidgets 7.6.3
json 2.0.9
numpy 1.19.5
matplotlib 3.1.1
astra 1.8.3
h5py 2.10.0
pyvista 0.33.3
pyevtk 1.2.0
scipy 1.1.0
skimage 0.15.0


## Reconstruction setup
Let's load in the pre-processed spray projected density images. For brevity we are only reconstructing the first frame of the spray, however the workflow remains unchanged for the other time steps. The rest of the frames are also made available in their raw format (they'll need to be [pre-processed](./pb02_preprocessing.ipynb)) [[1]](#References).

In [3]:
projRes_lo, projRes_hi = loadSpray(f'{out_fld}/spray_pbDens_frame0.hdf5')

We'll also be loading in time-averaged projected density maps for each of the Reynolds number conditions. These images will be used as a beam hardening correction for the time-resolved case.

In [4]:
projAvg_lo, projAvg_hi = loadSpray(f'{out_fld}/spray_pbDens_averaged.hdf5')

For reproducibility of the results in this work's corresponding paper, the geometry parameters utilized in the data processing for that work's results is used here instead of the outputs from [pb03_refinement.ipynb](pb03_refinement.ipynb). Due to the non-deterministic nature of the particle swarm optimization routine (it's a metaheuristic [[2]](#References)), the refined geometry parameters found in the previous step in this workflow might end up converging to a slightly different solution to within the defined constraints, however the improvement in the reconstructions is expected to be on a similar scale.

In [5]:
with open(f'{inp_fld}/geom_pso_paper.txt', 'r') as file:
     geom = json.load(file)

We'll be reconstructing the polychromatic beam data with a cone beam approach, as opposed to the monochromatic beam data which assumes a parallel beam. Let's start by loading in the system matrix using the refined geometry parameters, again constructed using the ASTRA toolbox [[3]](#References), [[4]](#References).

In [6]:
W = getSystemMatrix(geom)

As we will be utilizing an iterative reconstruction algorithm, we need to start out with a good initial volume guess. We'll be using the MLOS approach before that was introduced in the [refinement step](pb03_refinement.ipynb) [[5]](#References), [[6]](#References). This initial volume guess changes for each frame due to the dynamic nature of the spray$-$since we are only looking at the first time step (frame0) in this notebook, we only need to create one initial volume guess for the time-resolved case.

The constructed initial volumes are either binary (returned as floats) of 1s (liquid is expected to be present) or 0s (no liquid is expected to be present). This provides initial step only provides a structural guess of where liquid is expected to be present$-$the final reconstruction routine will refine the actual intensities.

In [7]:
# Initial volume for time-resolved case
volInit_lo, projInit_lo = MLOS(W, projRes_lo, 'proj', 3, fp='reconInit_lo')
volInit_hi, projInit_hi = MLOS(W, projRes_hi, 'proj', 3, fp='reconInit_hi')

# Initial volume for time-averaged case
volInitAvg_lo, projInitAvg_lo = MLOS(W, projAvg_lo, 'proj', 3, fp='reconAvgInit_lo')
volInitAvg_hi, projInitAvg_hi = MLOS(W, projAvg_hi, 'proj', 3, fp='reconAvgInit_hi')

We'll next want to create a "mask" volume that removes any extraneous regions on the exterior boundaries of the volume. This is still done through the `MLOS()` function, except now we will be constructing the volume by backprojecting arrays of 1s as opposed to backprojecting the initially measured spray images.

In [8]:
# Mask for time-resolved case
volMask_lo, _ = MLOS(W, projRes_lo, 'ones', 3, fp='reconMask_lo')
volMask_hi, _ = MLOS(W, projRes_hi, 'ones', 3, fp='reconMask_hi')

# Mask for time-averaged case
volMaskAvg_lo, _ = MLOS(W, projAvg_lo, 'ones', 3, fp='reconMaskAvg_lo')
volMaskAvg_hi, _ = MLOS(W, projAvg_hi, 'ones', 3, fp='reconMaskAvg_hi')

## Mass conservation check

Although a theoretical EPL calibration curve was used to convert the X-ray radiograph transmission values into quantitative path length values, further correction still needs to be made so that all views from the different X-ray tube sources give physically consistent values.

The first step will be to do an initial quick scalar correction. The jets before impingement are known to have a 2000 µm, so they are used to correct each of the three views. This correction is carried out in `scalarCF()` (see: [recon_utilities.py](./recon_utilities.py)).

In [9]:
# Correct the time-resolved case
projRes_lo = scalarCF(volInit_lo, projRes_lo, W, fp='scalarRes_lo')
projRes_hi = scalarCF(volInit_hi, projRes_hi, W, fp='scalarRes_hi')

# Correct the time-averaged case
projAvg_lo = scalarCF(volInitAvg_lo, projAvg_lo, W, fp='scalarAvg_lo')
projAvg_hi = scalarCF(volInitAvg_hi, projAvg_hi, W, fp='scalarAvg_hi')

The next step is to calculate the liquid masses captured from each of the views. As the images from the three lines of sight are synchronized and temporally resolved, the calculated masses should be consistent between all views. The mass is calculated by the summation of the projected density maps multiplied by the detector size squared, which is done in `massCheck()` (see: [recon_utilities.py](./recon_utilities.py)).

The first step is to mask off the projections to ensure all lines of sight are looking at the same parts of the spray. This is done by using the reprojections from the MLOS initialization routine as the mask.

In [10]:
# Mask off the time-resolved case
projRes_lo *= (projInit_lo > 0).astype(projRes_lo.dtype)
projRes_hi *= (projInit_hi > 0).astype(projRes_hi.dtype)

# Mask off the time-averaged case
projAvg_lo *= (projInitAvg_lo > 0).astype(projInitAvg_lo.dtype)
projAvg_hi *= (projInitAvg_hi > 0).astype(projInitAvg_hi.dtype)

Next, calculate the total liquid mass for each view.

In [11]:
# Calculate the mass and coefficient of variations for the time-resolved case
initMass_lo, cv_lo = massCheck(projRes_lo, geom, 'time-resolved, Re = 7,100')
initMass_hi, cv_hi = massCheck(projRes_hi, geom, 'time-resolved, Re = 10,600')

# Calculate the mass and coefficient of variations for the time-averaged case
initMassAvg_lo, cvAvg_lo = massCheck(
                                     projAvg_lo,
                                     geom,
                                     'time-averaged, Re = 7,100'
                                     )
initMassAvg_hi, cvAvg_hi = massCheck(
                                     projAvg_hi,
                                     geom,
                                     'time-averaged, Re = 10,600'
                                     )

Measured liquid masses for each view (time-resolved, Re = 7,100): [ 681031.  325636.  953151.] μg
Variation in liquid mass between views (time-resolved, Re = 7,100): 39.33%

Measured liquid masses for each view (time-resolved, Re = 10,600): [ 699175.  419010.  999509.] μg
Variation in liquid mass between views (time-resolved, Re = 10,600): 33.58%

Measured liquid masses for each view (time-averaged, Re = 7,100): [ 655070.  335199.  929322.] μg
Variation in liquid mass between views (time-averaged, Re = 7,100): 37.94%

Measured liquid masses for each view (time-averaged, Re = 10,600): [ 581376.  413203.  911318.] μg
Variation in liquid mass between views (time-averaged, Re = 10,600): 32.57%



This large variation in measured liquid masses between views (>30%) is attributed to the spatial beam hardening variation in the images in each view. Although a scalar correction was made based on the jets, a more rigorous spatial correction still needs to be done, i.e. an additional correction factor based on vertical (row) location in the image. Due to the polychromatic cone beam nature of the X-ray sources used here, as well as source-to-source variation, a spatial correction factor is to be expected. Here, an *ad hoc* approach is used where we will use reconstructions of the time-averaged projections to calculate the spatial correction factors for each view.

Let's start the correction by reconstructing the time-averaged projection data. We will use the `recon()` function (see: [recon_utilities.py](./recon_utilities.py)), which takes as inputs the system matrix `W`, the initial volume guess `initVol`, projection data `proj`, volume to be used for masking `mask`, lower and upper threshold bounds on the reconstructed volume intensities `bounds`, and maximum number of iterations `iters` and stopping criteria threshold `stop` that determine convergence. The iterative reconstruction algorithm used here is the maximum-likelihood expectation-maximization (MLEM) routine [[7]](#References), [[8]](#References). The function output is a tuple containing the reconstructed volume, reprojections of that volume, and history statistics that track the progression of the iterative reconstruction algorithm. For the mass conservation correction, only the reprojections are required here.

In [12]:
%%time

# Maximum ideal value in the reconstructed liquid density volumes [μg/mm^3]
maxVal = 1.314 * 1000

# Bounds to  use for thresholding the volume
# Lower threshold set to 2% of the maximum value (remove any noise) between 
# iterations, upper threshold set to the maximum value between iterations, 
# final lower threshold value set to 5% of the maximum value after iterations
bounds = (0.02*maxVal, maxVal, 0.05*maxVal)

# Maximum number of iterations
iters = 1000

# Stopping criteria: solution is stopped early if this threshold is met
# Chosen such that the change in total mass between iterations is <0.5 μg 
stop = 1E-4

# Reconstruct time-averaged projections
_, reprojAvg_lo, _ = recon(
                           W,
                           volInitAvg_lo,
                           projAvg_lo,
                           volMaskAvg_lo,
                           bounds,
                           iters,
                           stop,
                           fp='reconAvg_lo'
                           )
_, reprojAvg_hi, _ = recon(
                           W,
                           volInitAvg_hi,
                           projAvg_hi,
                           volMaskAvg_hi,
                           bounds,
                           iters,
                           stop,
                           fp='reconAvg_hi'
                           )

Wall time: 2min 25s


Next, the spatial correction factor mappings are calculated by taking the ratio between the time-averaged reprojection and projection datasets. After ratio calculation, a vertical profile is calculated by finding the mean ratio value for each row in the image. This is done so that any differences between the time-resolved and time-averaged case does not negatively bias the time-resolved reconstructions (e.g. an isolated droplet on the edge that shows up in the time-resolved case may not be present in the time-averaged case and could be zeroed out). Refer to `spatialCF()` for further detail (see: [recon_utilities.py](./recon_utilities.py)).

The spatial corrections are calculated for each Reynolds number case separately to account for any differences in path length distributions.

In [13]:
# Calculate spatial correction factors using the time-averaged projections
spatialCF_lo = spatialCF(projAvg_lo, reprojAvg_lo)
spatialCF_hi = spatialCF(projAvg_hi, reprojAvg_hi)

Now let's apply the spatial correction to the time-resolved case.

In [14]:
# Apply spatial correction factors
projCorr_lo = projRes_lo * spatialCF_lo
projCorr_hi = projRes_hi * spatialCF_hi

Let's verify the effectiveness by calculating mass conservation checks again on the corrected datasets.

In [15]:
# Calculate the mass and coefficient of variations for the time-resolved case
corrMass_lo, cvCorr_lo = massCheck(
                                   projCorr_lo,
                                   geom,
                                   'time-resolved, Re = 7,100'
                                   )
corrMass_hi, cvCorr_hi = massCheck(
                                   projCorr_hi,
                                   geom,
                                   'time-resolved, Re = 10,600'
                                   )

Measured liquid masses for each view (time-resolved, Re = 7,100): [ 553107.  599733.  739043.] μg
Variation in liquid mass between views (time-resolved, Re = 7,100): 12.53%

Measured liquid masses for each view (time-resolved, Re = 10,600): [ 631041.  574185.  737155.] μg
Variation in liquid mass between views (time-resolved, Re = 10,600): 10.43%



## Time-resolved reconstruction

In [16]:
%%time

# Reconstruct time-averaged projections
[volRes_lo, reprojRes_lo, histRes_lo] = recon(
                                              W,
                                              volInit_lo,
                                              projCorr_lo,
                                              volMask_lo,
                                              bounds,
                                              iters,
                                              stop,
                                              fp='reconRes_lo'
                                              )
[volRes_hi, reprojRes_hi, histRes_hi] = recon(
                                              W,
                                              volInit_hi,
                                              projCorr_hi,
                                              volMask_hi,
                                              bounds,
                                              iters,
                                              stop,
                                              fp='reconRes_hi'
                                              )

Wall time: 1min 16s


Let's run the mass conservation check again on these final reconstructions just to be safe.

In [17]:
# Calculate the mass and coefficient of variations for the time-resolved case
reconMass_lo, cvRecon_lo = massCheck(
                                     reprojRes_lo,
                                     geom,
                                     'time-resolved, Re = 7,100'
                                     )
reconMass_hi, cvRecon_hi = massCheck(
                                     reprojRes_hi,
                                     geom,
                                     'time-resolved, Re = 10,600'
                                     )

Measured liquid masses for each view (time-resolved, Re = 7,100): [ 528643.  586911.  718456.] μg
Variation in liquid mass between views (time-resolved, Re = 7,100): 12.99%

Measured liquid masses for each view (time-resolved, Re = 10,600): [ 534814.  594588.  726013.] μg
Variation in liquid mass between views (time-resolved, Re = 10,600): 12.91%



We can also inspect the solution history per iteration for each of the reconstructions. The errors between the projection and reprojections are tabulated as the NRMSE (normalized root-mean-squared error in intensity), liquid mass, PSNR (peak signal-to-noise ratio), and SSIM (structural similarity index measure). The convergence is tabulated as the change in the $L_2$-norm in the projected views between iterations. The solution is run until all views reach a change in the $L_2$-norm of less than the equivalent of ~0.5 μg or until 1000 iterations is reached, whichever comes first. There is a small jump at the last iteration due to the final, slightly more aggressive thresholding done on the entire volume.

In [18]:
interact(
         showHistory, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         hist=fixed((histRes_lo, histRes_hi)),
         Re=fixed((7100, 10600)),
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Out…

## Visualization

We can visualize our results by first viewing the (mass-corrected) measured projections against the reconstructed volume's reprojections for each camera. This is done through the `compareProj()` function below (see: [recon_utilities.py](./recon_utilities.py)). We see that the reprojections on the right-hand side closely matches the left-hand side measured projections, with the main differences being that the reprojections seem slightly more "blurrier", which is to be expected as a smoothing filter was used between iterations to aid in convergence.

In [19]:
interact(
         compareProj, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         proj=fixed((projCorr_lo, projCorr_hi)),
         reproj=fixed((reprojRes_lo, reprojRes_hi)),
         Re=fixed((7100, 10600)),
         view=widgets.Dropdown(options=[(1,0), (2,1), (3,2)],
                               description='Camera',
                               ),
         cmap=widgets.Dropdown(options=plt.colormaps(),
                               description='Colormap', 
                               value='Blues')
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Dro…

The accompanying paper to this work utilizes *ParaView* [[9]](#References) for volume visualization. We first need to output the volumes as a VTI file using the Visualization Toolkit (VTK) standard [[10]](#References). This is an open-source software that allows for 3D visualization through a multitude of tools, including *ParaView* which is developed with VTK in mind. The processing is done in `convertVTK()` (see: [recon_utilities.py](./recon_utilities.py)), which crops down the volume, centers the data on the impingement point, and converts the values into liquid volume fraction (LVF) scaled to the range 0-1000. The impingement point was manually found by axially slicing through the volume in *ParaView* to locate the point at which the two jets fully intersect.

In [20]:
# Convert and save volumes
convertVTK(volRes_lo, W, 'volume_Re-7100')
convertVTK(volRes_hi, W, 'volume_Re-10600')

These files can now be imported directly into *ParaView*, where more advanced visualization can be done.

We can also visualize the volume as well in Jupyter using the `pyvista` library. As a quick demonstration of the reconstruction results, let's interactively visualize some selected isosurfaces here using `showVolume()` (see: [recon_utilities.py](./recon_utilities.py)). The figure can be manipulated through zooming (hold right-click and drag or scroll in/out), panning (hold shift and drag left-click), and rotating (drag left-click). For further visualization, such as slicing through the volumes, refer to the [pyvista documentation](https://docs.pyvista.org) for some inspiration (or use *ParaView*!). 

*Note*: this figure may not be viewable in Jupyter Lab$-$I would recommend using Jupyter Notebook instead.

In [21]:
interact(
         showVolume, 
         state=widgets.ToggleButtons(options=[(7100, 0), (10600, 1)], 
                                     description='Reynolds No.',
                                     ),
         vol=fixed(('volume_Re-7100', 'volume_Re-10600')),
         Re=fixed((7100, 10600)),
         );

interactive(children=(ToggleButtons(description='Reynolds No.', options=((7100, 0), (10600, 1)), value=0), Out…

## Next steps

This concludes the polychromatic beam reconstruction workflow! Further data analysis on the volume datasets can be done on the NumPy arrays (`volRes_lo`, `volRes_hi`) directly in Python, with simple visualization of the volume done through software such as `pyvista` in Python or through more advanced external tools such as *ParaView* (recommended).

## References

1. Rahman, N., Meyer, T. Quantitative polychromatic 4D and monochromatic 2D X-ray tomography of liquid jet breakup dynamics. *Purdue University Research Repository* (2022). doi: [10.4231/G20X-4Z27](https://doi.org/10.4231/G20X-4Z27).

2. Abdel-Basset, M., Abdel-Fatah, L. & Sangaiah, A. K. Metaheuristic Algorithms: A Comprehensive Review. in *Computational Intelligence for Multimedia Big Data on the Cloud with Engineering Applications* 185–231 (Elsevier, 2018). doi: [10.1016/B978-0-12-813314-9.00010-4](https://doi.org/10.1016/B978-0-12-813314-9.00010-4).

3. van Aarle, W. et al. The ASTRA Toolbox: A platform for advanced algorithm development in electron tomography. *Ultramicroscopy* 157, 35–47 (2015). doi: [10.1016/j.ultramic.2015.05.002](https://doi.org/10.1016/j.ultramic.2015.05.002).

4. van Aarle, W. et al. Fast and flexible X-ray tomography using the ASTRA toolbox. *Opt. Express* 24, 25129 (2016). doi: [10.1364/OE.24.025129](https://doi.org/10.1364/OE.24.025129).

5. Atkinson, C. & Soria, J. An efficient simultaneous reconstruction technique for tomographic particle image velocimetry. *Exp. Fluids* 47, 553–568 (2009). doi: [10.1007/s00348-009-0728-0](https://doi.org/10.1007/s00348-009-0728-0).

6. Worth, N. A. & Nickels, T. B. Acceleration of Tomo-PIV by estimating the initial volume intensity distribution. *Exp. Fluids* 45, 847–856 (2008). doi: [10.1007/s00348-008-0504-6](https://doi.org/10.1007/s00348-008-0504-6).

7. Jingyu Cui, Pratx, G., Bowen Meng & Levin, C. S. Distributed MLEM: An Iterative Tomographic Image Reconstruction Algorithm for Distributed Memory Architectures. *IEEE Trans. Med. Imaging* 32, 957–967 (2013). doi: [10.1109/TMI.2013.2252913](https://doi.org/10.1109/TMI.2013.2252913).

8. Slomski, A. et al. 3D PET image reconstruction based on the maximum likelihood estimation method (MLEM) algorithm. *Bio-Algorithms and Med-Systems* 10, 1–7 (2014). doi: [10.1515/bams-2013-0106](https://doi.org/10.1515/bams-2013-0106).

7. *ParaView*. Available at: http://www.paraview.org/.  

8. Schroeder, W., Martin, K. & Lorensen, B. *The Visualization Toolkit*. (Kitware, Inc, 2006).

---

**Author**: Naveed Rahman, Purdue University, 23 March 2022

---